In [84]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv(".\\adult.csv")
data.info()
data = data.drop((data[data["native-country"] == "Holand-Netherlands"]).index)
# Fiz esse drop porque originalmente só tem 1 registro com esse native-country, ficava dando problema se ele não fosse pro treino então preferi retirá-lo
data = data.drop(data[(data['race'] != 'White') & (data['race'] != "Black")].index)

data["race"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


White                 41761
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: race, dtype: int64

In [85]:
x = data.drop(["income", "race"], axis = 1)
Y = data["race"]

from sklearn.model_selection import train_test_split

x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size = 0.3)

In [86]:
from sklearn.preprocessing import OrdinalEncoder

categoricalLabels = (x_train.select_dtypes(include = ['object'])).columns

encoder = OrdinalEncoder(dtype = 'int32')

num_x_train = x_train.copy()
num_x_test = x_test.copy()

num_x_train[categoricalLabels] = encoder.fit_transform(x_train[categoricalLabels])
print(num_x_test)
num_x_test[categoricalLabels] = encoder.transform(x_test[categoricalLabels])

num_Y_test = Y_test.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 1, "Amer-Indian-Eskimo": 1, "Other": 4})
num_Y_train = Y_train.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 1, "Amer-Indian-Eskimo": 1, "Other": 4})
print()

       age     workclass  fnlwgt     education  educational-num  \
5633    29       Private  115677          11th                7   
31747   30  Self-emp-inc  173858  Some-college               10   
42737   36       Private  343476       HS-grad                9   
40037   34       Private  345705     Bachelors               13   
42328   41     State-gov   33126     Bachelors               13   
...    ...           ...     ...           ...              ...   
6891    62     Local-gov  242341  Some-college               10   
7788    28       Private  269317       HS-grad                9   
41637   45       Private  102308       Masters               14   
33554   57   Federal-gov  135028       HS-grad                9   
43627   41       Private  242089       Masters               14   

           marital-status         occupation    relationship  gender  \
5633        Never-married  Machine-op-inspct       Own-child    Male   
31747  Married-civ-spouse    Exec-managerial       

## Classificador da Raça de uma Pessoa
A acurácia é de 0.8 mais ou menos, quando adicionamos excluímos os registros com raça "Other". A acurácia considerando "Other" é de 0.75, variando pouco mais 0.01.

O que demonstra, que com uma acurácia relativamente alta, podemos prever a Raça de uma pessoa utilizando apenas as correlações entre ela e os atributos. 

In [87]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

modelo = DecisionTreeClassifier()

modelo.fit(num_x_train, num_Y_train)
previsao = modelo.predict(num_x_test)

acuraciaNP = mean_absolute_error(num_Y_test, previsao)
print(acuraciaNP)

0.25319047294069474


erro em média de 0.24 para quando eu faço Brancos e Outras Etnias
erro em média de 0.16 para quando eu faço Brancos e Negros, apagando as outras etnias além dessas duas